In [1]:
pip install  textBlob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 5.2 MB/s 
     |████████████████████████████████| 287 kB 40.5 MB/s 


In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

from wordcloud import WordCloud

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import contractions
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import nltk
nltk.download('punkt')
tokenizer = RegexpTokenizer(r'\w+')

def tokenize_text(text):
    text_processed = " ".join(tokenizer.tokenize(text))
    return text_processed

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import en_core_web_sm
import nltk
nltk.download('punkt')
nlp = en_core_web_sm.load(disable=['parser', 'tagger', 'ner'])

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))
    lemmatized_text_list = list()
    
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'a')) # Lemmatise adjectives. Not doing anything since we remove all adjective
        elif tag.startswith('V'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'v')) # Lemmatise verbs
        elif tag.startswith('N'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'n')) # Lemmatise nouns
        elif tag.startswith('R'):
            lemmatized_text_list.append(lemmatizer.lemmatize(word,'r')) # Lemmatise adverbs
        else:
            lemmatized_text_list.append(lemmatizer.lemmatize(word)) # If no tags has been found, perform a non specific lemmatisation
    
    return " ".join(lemmatized_text_list)

"""def lemmatize_text(text):
    lemmatized_text_list = list()

    for word in text.split():
        lemmatized_text_list.append(lemmatizer.lemmatize(word, "v")) # Lemmatise verbs

    return " ".join(lemmatized_text_list)"""

"""def lemmatize_text(text):
    
    doc = nlp.pipe(text, batch_size=32, n_process=8)
    
    return " ".join([w.lemma_ if w.lemma_ != '-PRON-' else w.lower_ for w in doc])"""

    #return " ".join([token.lemma_ for token in doc])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser', 'tagger', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


'def lemmatize_text(text):\n    \n    doc = nlp.pipe(text, batch_size=32, n_process=8)\n    \n    return " ".join([w.lemma_ if w.lemma_ != \'-PRON-\' else w.lower_ for w in doc])'

In [6]:
import nltk
nltk.download('punkt')
def normalize_text(text):
    return " ".join([word.lower() for word in text.split()])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
def contraction_text(text):
    return contractions.fix(text)

In [8]:
negative_words = ['not', 'no', 'never', 'nor', 'hardly', 'barely']
negative_prefix = "NOT_"

In [9]:
def get_negative_token(text):
    tokens = text.split()
    negative_idx = [i+1 for i in range(len(tokens)-1) if tokens[i] in negative_words]
    for idx in negative_idx:
        if idx < len(tokens):
            tokens[idx]= negative_prefix + tokens[idx]
    
    tokens = [token for i,token in enumerate(tokens) if i+1 not in negative_idx]
    
    return " ".join(tokens)

In [10]:
from spacy.lang.en.stop_words import STOP_WORDS

def remove_stopwords(text):
    english_stopwords = stopwords.words("english") + list(STOP_WORDS) + ["tell", "restaurant"]
    
    return " ".join([word for word in text.split() if word not in english_stopwords])

In [11]:
def preprocess_text(text):
    
    # Tokenize review
    text = tokenize_text(text)
    
    # Lemmatize review
    text = lemmatize_text(text)
    
    # Normalize review
    text = normalize_text(text)
    
    # Remove contractions
    text = contraction_text(text)

    # Get negative tokens
    text = get_negative_token(text)
    
    # Remove stopwords
    text = remove_stopwords(text)
    
    return text

## Jalon 3

In [12]:
import pickle
file_name="/content/drive/MyDrive/TP-NLP/Jalon3/vectorizer.bin"
with (open(file_name, "rb")) as f:
    while True:
        try:
            vectorizer=pickle.load(f)
        except EOFError:
            break
vectorizer

TfidfVectorizer(max_df=0.7, min_df=0.01)

In [13]:
import pickle
file_name="/content/drive/MyDrive/TP-NLP/Jalon3/model_nmf.bin"

with (open(file_name, "rb")) as f:
    while True:
        try:
            nmf_model=pickle.load(f)
        except EOFError:
            break
nmf_model

NMF(n_components=15)

In [14]:
text=preprocess_text("I am good")
text

'good'

In [15]:
import pandas as pd
liste=[]
liste.append("I am good")
x=vectorizer.transform(liste)
y=nmf_model.transform(x)
print(y)
ind=[]
argsort=np.argsort(y, axis=1)
print(argsort)
for i in range(2):   
      ind.append(argsort[0][len(argsort[0])-(i+1)])
ind




[[0.00000000e+00 0.00000000e+00 4.60482251e-05 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.29701667e-01]]
[[ 0  1  3  4  5  6  7  8  9 10 11 12 13  2 14]]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NMF was fitted with feature names
  "X does not have valid feature names, but"


[14, 2]

In [18]:
#Fonction de prediction

from textblob import TextBlob
import pandas as pd
interpretation_topi={
    'topic0' : 'TICKET PAYEMENT',
    'topic1' : 'OVERLOADED RESTAURANT',
    'topic2' : 'PIZZA/DELIVERY',
    'topic3' : 'PRICES/ONLINE-ORDERING',
    'topic4' : 'FOOD/SERVICE',
    'topic5' : 'TABLE/BOTHERING',
    'topic6' : 'SERVICE',
    'topic7' : 'TEMPS D ATTENTE',
    'topic8' : 'CHICKEN',
    'topic9' : 'BEER/BAR',
    'topic10' : 'BAD PLACE TO EAT',
    'topic11' : 'SUSHI',
    'topic12' : 'LUNCH/SANDWICH',
    'topic13' : 'AMBIANCE',
    'topic14' : 'WAITER'
}
def prediction_text(vectorizer,nmf_model,n_topic,text):
  topic_list=['TICKET PAYEMENT','OVERLOADED RESTAURANT','PIZZA/DELIVERY','PRICES/ONLINE-ORDERING','FOOD/SERVICE','TABLE/BOTHERING','SERVICE','TEMPS D ATTENTE','CHICKEN', 'BEER/BAR','BAD PLACE TO EAT','SUSHI',
     'LUNCH/SANDWICH',
     'AMBIANCE',
     'WAITER']

  txt=TextBlob(text)
  polarity=txt.polarity
  liste=[]
  text_cleaned=preprocess_text(text)
  liste.append(text_cleaned)
  idx_l=[]
  if (polarity<0):
    text_cleaned=preprocess_text(text)
    liste.append(text_cleaned)
    x=vectorizer.transform(liste)
    y=nmf_model.transform(x)
    ind=[]
    argsort=np.argsort(y, axis=1)
    print(argsort)
    for i in range(n_topic):   
      ind.append(argsort[0][len(argsort[0])-(i+1)])
    
    for i in ind:
      idx_l.append(topic_list[i])

  return polarity,idx_l


    


In [21]:
p,l=prediction_text(vectorizer,nmf_model,2,"I am bad")
print("polarity is : ",p)
print("Topics are : ",l)


[[ 0  3  5  6  7  8  9 10 11 14  2  1 12 13  4]
 [ 0  3  5  6  7  8  9 10 11 14  2  1 12 13  4]]
polarity is :  -0.6999999999999998
Topics are :  ['FOOD/SERVICE', 'AMBIANCE']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but NMF was fitted with feature names
  "X does not have valid feature names, but"
